In [ ]:
# 1ST ARCHITECTURE

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 10

# CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Autoencoder with Classification Head
class AutoencoderWithClassifier(nn.Module):
    def __init__(self):
        super(AutoencoderWithClassifier, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  # (64, 16, 16)
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # (128, 8, 8)
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # (256, 4, 4)
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  # (128, 8, 8)
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # (64, 16, 16)
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  # (3, 32, 32)
            nn.Tanh()  # Normalize to [-1, 1]
        )
        # Classification Head
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512),  # Flattened size: 256 * 4 * 4
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)  # 10 classes for CIFAR-10
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        logits = self.classifier(encoded.view(encoded.size(0), -1))  # Flatten before classifier
        return decoded, logits




model = AutoencoderWithClassifier().to(device)

reconstruction_loss = nn.MSELoss()
classification_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

correct_classwise = torch.zeros(10)  # To count correct predictions per class
total_classwise = torch.zeros(10)    # To count total samples per class

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_correct = 0
    total_samples = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        decoded, logits = model(images)
        loss_recon = reconstruction_loss(decoded, images)
        loss_class = classification_loss(logits, labels)
        loss = loss_recon + loss_class  # Combined loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        _, predicted = torch.max(logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Count accuracy per class
        for i in range(len(labels)):
            total_classwise[labels[i]] += 1
            correct_classwise[labels[i]] += (predicted[i] == labels[i]).item()
    
    train_accuracy = total_correct / total_samples * 100
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Testing loop
model.eval()
total_correct = 0
total_samples = 0
reconstruction_losses = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        decoded, logits = model(images)

        # Compute classification accuracy
        _, predicted = torch.max(logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Compute reconstruction loss
        loss_recon = reconstruction_loss(decoded, images)
        reconstruction_losses.append(loss_recon.item())

test_accuracy = total_correct / total_samples * 100
average_reconstruction_loss = sum(reconstruction_losses) / len(reconstruction_losses)
print(f"Test Accuracy: {test_accuracy:.2f}%, Average Reconstruction Loss: {average_reconstruction_loss:.4f}")

# Visualizing original and reconstructed images
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        decoded, _ = model(images)
        break

fig, axs = plt.subplots(2, 8, figsize=(12, 4))
for i in range(8):
    axs[0, i].imshow(np.transpose((images[i].cpu().numpy() * 0.5 + 0.5), (1, 2, 0)))  # Denormalize
    axs[1, i].imshow(np.transpose((decoded[i].cpu().numpy() * 0.5 + 0.5), (1, 2, 0)))
    axs[0, i].axis('off')
    axs[1, i].axis('off')
plt.show()


class_accuracies = correct_classwise / total_classwise * 100
# Print the accuracy for each class
for i in range(10):
    print(f'Accuracy for class {train_dataset.classes[i]}: {class_accuracies[i]:.2f}%')


In [ ]:
# 2ND ARCHITECTURE

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 10

# CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Autoencoder with Classification Head
class AutoencoderWithClassifier(nn.Module):
    def __init__(self):
        super(AutoencoderWithClassifier, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1), 
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1), 
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=1),    
            nn.Tanh()
        )
        # Classification Head
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 512),  # Flattened size: 256 * 4 * 4
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)  # 10 classes for CIFAR-10
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        logits = self.classifier(encoded.view(encoded.size(0), -1))  # Flatten before classifier
        return decoded, logits


model = AutoencoderWithClassifier().to(device)

reconstruction_loss = nn.MSELoss()
classification_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_correct = 0
    total_samples = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        decoded, logits = model(images)
        loss_recon = reconstruction_loss(decoded, images)
        loss_class = classification_loss(logits, labels)
        loss = loss_recon + loss_class  # Combined loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track accuracy
        _, predicted = torch.max(logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
    
    train_accuracy = total_correct / total_samples * 100
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Testing loop
model.eval()
total_correct = 0
total_samples = 0
reconstruction_losses = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        decoded, logits = model(images)

        # Compute classification accuracy
        _, predicted = torch.max(logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Compute reconstruction loss
        loss_recon = reconstruction_loss(decoded, images)
        reconstruction_losses.append(loss_recon.item())

test_accuracy = total_correct / total_samples * 100
average_reconstruction_loss = sum(reconstruction_losses) / len(reconstruction_losses)
print(f"Test Accuracy: {test_accuracy:.2f}%, Average Reconstruction Loss: {average_reconstruction_loss:.4f}")

# Visualizing original and reconstructed images
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        decoded, _ = model(images)
        break

fig, axs = plt.subplots(2, 8, figsize=(12, 4))
for i in range(8):
    axs[0, i].imshow(np.transpose((images[i].cpu().numpy() * 0.5 + 0.5), (1, 2, 0)))  # Denormalize
    axs[1, i].imshow(np.transpose((decoded[i].cpu().numpy() * 0.5 + 0.5), (1, 2, 0)))
    axs[0, i].axis('off')
    axs[1, i].axis('off')
plt.show()


In [ ]:
# 3RD ARCHITECTURE

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 10

# CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Autoencoder with Classification Head
class AutoencoderWithClassifier(nn.Module):
    def __init__(self):
        super(AutoencoderWithClassifier, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # (64, 32, 32)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # (128, 16, 16)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # (256, 8, 8)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),  # (512, 4, 4)
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),  # (256, 8, 8)
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # (128, 16, 16)
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   # (64, 32, 32)
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=1),     # (3, 32, 32)
            nn.Tanh()
        )

        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 512),  # Flattened size: 512 * 4 * 4
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)  # 10 classes for CIFAR-10
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        logits = self.classifier(encoded.view(encoded.size(0), -1))  # Flatten before classifier
        return decoded, logits



model = AutoencoderWithClassifier().to(device)

reconstruction_loss = nn.MSELoss()
classification_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_correct = 0
    total_samples = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        decoded, logits = model(images)
        loss_recon = reconstruction_loss(decoded, images)
        loss_class = classification_loss(logits, labels)
        loss = loss_recon + loss_class  # Combined loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track accuracy
        _, predicted = torch.max(logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
    
    train_accuracy = total_correct / total_samples * 100
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Testing loop
model.eval()
total_correct = 0
total_samples = 0
reconstruction_losses = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        decoded, logits = model(images)

        # Compute classification accuracy
        _, predicted = torch.max(logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Compute reconstruction loss
        loss_recon = reconstruction_loss(decoded, images)
        reconstruction_losses.append(loss_recon.item())

test_accuracy = total_correct / total_samples * 100
average_reconstruction_loss = sum(reconstruction_losses) / len(reconstruction_losses)
print(f"Test Accuracy: {test_accuracy:.2f}%, Average Reconstruction Loss: {average_reconstruction_loss:.4f}")

# Visualizing original and reconstructed images
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        decoded, _ = model(images)
        break

fig, axs = plt.subplots(2, 8, figsize=(12, 4))
for i in range(8):
    axs[0, i].imshow(np.transpose((images[i].cpu().numpy() * 0.5 + 0.5), (1, 2, 0)))  # Denormalize
    axs[1, i].imshow(np.transpose((decoded[i].cpu().numpy() * 0.5 + 0.5), (1, 2, 0)))
    axs[0, i].axis('off')
    axs[1, i].axis('off')
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

# Get a batch of data for PCA
data_iter = iter(train_loader)
images, labels = next(data_iter)

# Convert images to numpy array for PCA
images_np = images.view(images.size(0), -1).numpy()


# PCA
n_components = 40  
pca = PCA(n_components=n_components)
images_pca = pca.fit_transform(images_np)  
images_reconstructed = pca.inverse_transform(images_pca)  

# Reshape reconstructed images back to original shape
images_reconstructed = torch.tensor(images_reconstructed).view(-1, 3, 32, 32)

# Visualize results
def plot_images(original, reconstructed, n=5):
    plt.figure(figsize=(12, 5))
    for i in range(n):
        # Original
        plt.subplot(2, n, i + 1)
        plt.imshow(original[i].permute(1, 2, 0) * 0.5 + 0.5)
        plt.axis('off')
        if i == n // 2:
            plt.title('Original Images')

        # Reconstructed
        plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed[i].permute(1, 2, 0).detach().numpy() * 0.5 + 0.5)
        plt.axis('off')
        if i == n // 2:
            plt.title('Reconstructed Images')
    plt.tight_layout()
    plt.show()

# Visualize some examples
plot_images(images, images_reconstructed)


# PCA transformation of the full dataset (training set)
def transform_data_with_pca(data_loader):
    pca_data = []
    labels_data = []
    for images, labels in data_loader:
        images_np = images.view(images.size(0), -1).numpy()
        images_pca = pca.transform(images_np) 
        pca_data.append(torch.tensor(images_pca))
        labels_data.append(labels)
    
    # Combine all batches
    pca_data = torch.cat(pca_data, dim=0)
    labels_data = torch.cat(labels_data, dim=0)
    
    return pca_data, labels_data

# Apply PCA transformation to train and test data
train_data_pca, train_labels = transform_data_with_pca(train_loader)
test_data_pca, test_labels = transform_data_with_pca(test_loader)

# Classifier neural network
class ClassifierNN(nn.Module):
    def __init__(self):
        super(ClassifierNN, self).__init__()
        self.fc1 = nn.Linear(n_components, 512)  # Input size is PCA components after the transform
        self.fc2 = nn.Linear(512, 10)  # 10 output classes for CIFAR-10
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = ClassifierNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert PCA-transformed data to device tensors
train_data_pca = train_data_pca.to(device)
train_labels = train_labels.to(device)
test_data_pca = test_data_pca.to(device)
test_labels = test_labels.to(device)



correct_classwise = torch.zeros(10)  # To count correct predictions per class
total_classwise = torch.zeros(10)    # To count total samples per class
 
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for i in range(0, len(train_data_pca), 100):
        images = train_data_pca[i:i+100]
        labels = train_labels[i:i+100]

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Count accuracy per class
        for i in range(len(labels)):
            total_classwise[labels[i]] += 1
            correct_classwise[labels[i]] += (predicted[i] == labels[i]).item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_data_pca):.4f}, Accuracy: {(correct * 100) / total:.2f}%")


# Evaluate on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for i in range(0, len(test_data_pca), 100):
        images = test_data_pca[i:i+100]
        labels = test_labels[i:i+100]
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {(correct * 100) / total:.2f}%")

class_accuracies = correct_classwise / total_classwise * 100
# Print the accuracy for each class
for i in range(10):
    print(f'Accuracy for class {trainset.classes[i]}: {class_accuracies[i]:.2f}%')
